<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [76]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import re
from function_library import *
from function_library2 import *

In [77]:
# namelist= ['Marks','Dailey','Desloge','Miller','Mustian','Sauls','Ziffer','Gillum','Maddox','Lindley','Dozier','Proctor','Richardson']
# for i in range(len(namelist)):
#     namelist[i] = namelist[i].lower()
# keep_idx=[]
# for i in range(len(from_list)):
#     for name in namelist:
#         if name in from_list[i].lower() or name in to_list[i].lower() or name in cc_list[i].lower():
#             keep_idx.append(i)
#             break

# len(keep_idx)

In [78]:
re_behalf = re.compile(r'(.*)[Oo]n [Bb]ehalf')
re_bracket = re.compile(r'(.*)[\[\<\(]mailto\:(.*)[\]\>\)]')
re_bracket2 = re.compile(r'(.*)[\[\<\(](.*)[\]\>\)]')

re_name1 = re.compile(r'.*?([A-Za-z]+\s?[A-Z]?\.?)\s([A-Za-z]+)')
re_name2 = re.compile(r'.*?([A-Za-z]+),\s([A-Za-z]+\s?[A-Z]?\.?)')
re_email = re.compile(r'.*?([a-zA-Z_]*\.?\w+@[a-zA-Z_0-9]*\.?[a-zA-Z_]*\.?[a-zA-Z]{2,3})')


In [79]:
# either the sender or the recipient
# df = pd.read_csv('new_clean_output.csv',index_col = 0)
df = pd.read_csv('old_output.csv')


df = df.drop_duplicates(keep='first',subset=['Sent'])
df = df.reset_index(drop=True)

from_list = df['From'].values.tolist()
to_list = df['To'].values.tolist()
cc_list = df['CC'].values.tolist()

try:
    del df['title_length_c']
    del df['title_length_w']
    del df['body_length_w']
    del df['POS']
    del df['POS_dict']
except:
    pass


In [80]:
# test
sss = 'McGee, Michele'.lower()
sss1 = 'sarah_aviles@85cbakerycafe.com'
sss2 = 'Elias Mathes TDIA'
sss3 = 'Adam B Corey'
print(re_name2.findall(sss))

sss4 = 'Jim Murdaugh Ph. D. (murdaugj@tcc.fl.edu); Will Butler (w.f.butler@embarqm.com)'

for t in sss4.split(';'):
    print(t)
    if re_bracket2.match(t):
        print('match')

[('mcgee', 'michele')]
Jim Murdaugh Ph. D. (murdaugj@tcc.fl.edu)
match
 Will Butler (w.f.butler@embarqm.com)
match


In [81]:
people_list = []
named_email_list = []
name_list = []
# find unique complete person in From section

for f in from_list: 
    f = f[2:-2].lower()
    if re_behalf.match(f):
        f = re_behalf.findall(f)[0]
    if re_bracket.match(f):
        tname, temail = re_bracket.findall(f)[0]
        tname=tname.lower().strip()
        temail=temail.lower()
        
#         print(tname, temail)
        if re_email.match(temail) and re_email.match(tname) == None:
            email = re_email.findall(temail)[0]
            if len(tname.split()) != 2:
                first_name = tname
                last_name = ' '
            else:
                if re_name1.match(tname):
                    name = re_name1.findall(tname)[0]
                    first_name = name[0]
                    last_name = name[1]
                elif re_name2.match(tname): 
                    name = re_name2.findall(tname)[0]
                    first_name = name[1]
                    last_name = name[0]
                else:
                    print('error: cannot find name in tname')
                    print('tname:',tname)
                    break
            if email not in named_email_list:
                person = (first_name, last_name, email)
                people_list.append(person)
                named_email_list.append(email)
                name_list.append(first_name + ' ' + last_name)
            else:
#                 print(person)
                pass

error: cannot find name in tname
tname: jeﬀ barbacci


In [82]:
len(people_list)

551

In [83]:
# find unique complete person in TO section
for ts in to_list: 
    ts = ts[2:-2].lower()
    ts = ts.split(';')
    for t in ts:
        t = t.strip("'")

        if re_bracket.match(t) or re_bracket2.match(t):
            if re_bracket.match(t):
                tname, temail = re_bracket.findall(t)[0]
            else:
                tname, temail = re_bracket2.findall(t)[0]
            tname=tname.lower().strip()
            temail=temail.lower()
            if tname == '':
                continue

            if re_email.match(temail) and re_email.match(tname) == None:
                email = re_email.findall(temail)[0]
                if len(tname.split()) != 2:
                    first_name = tname
                    last_name = ' '
                else:
                    if re_name1.match(tname):
                        name = re_name1.findall(tname)[0]
                        first_name = name[0]
                        last_name = name[1]
                    elif re_name2.match(tname): 
                        name = re_name2.findall(tname)[0]
                        first_name = name[1]
                        last_name = name[0]
                    else:
                        break
                        print('error: cannot find name in tname')
                        print('tname:',tname)
                if email not in named_email_list:
                    person = (first_name, last_name, email)
                    people_list.append(person)
                    named_email_list.append(email)
                    name_list.append(first_name + ' ' + last_name)
                    
#                     print(person)
#                     print(tname)
#                     print(t)
                
                else:
#                     print(person)
                    pass

In [84]:
len(people_list)

2069

In [85]:
# find unique complete person in CC section
for ccs in cc_list: 
    ccs = ccs[2:-2].lower()
    ccs = ccs.split(';')
    for cc in ccs:
        cc = cc.strip("'")

        if re_bracket.match(cc) or re_bracket2.match(cc):
            if re_bracket.match(cc):
                tname, temail = re_bracket.findall(cc)[0]
            else:
                tname, temail = re_bracket2.findall(cc)[0]
            tname=tname.lower().strip()
            temail=temail.lower()
            if tname == '':
                continue

            if re_email.match(temail) and re_email.match(tname) == None:
                email = re_email.findall(temail)[0]
                if len(tname.split()) != 2:
                    first_name = tname
                    last_name = ' '
                else:
                    if re_name1.match(tname):
                        name = re_name1.findall(tname)[0]
                        first_name = name[0]
                        last_name = name[1]
                    elif re_name2.match(tname): 
                        name = re_name2.findall(tname)[0]
                        first_name = name[1]
                        last_name = name[0]
                    else:
                        break
                        print('error: cannot find name in tname')
                        print('tname:',tname)
                if email not in named_email_list:
                    person = (first_name, last_name, email)
                    people_list.append(person)
                    named_email_list.append(email)
                    name_list.append(first_name + ' ' + last_name)
                    
#                     print(person)
#                     print(tname)
#                     print(cc)
                
                else:
#                     print(person)
                    pass

In [86]:
len(people_list)

2361

In [87]:
new_from_list = []

unknown_idx = 0
# replace the From section with unique people information

for f in from_list: 
    email_exist_flag = 0
    f = f[2:-2].lower().strip("'")
    email = ''
    first_name = ''
    last_name = ''
    if re_behalf.match(f):
        f = re_behalf.findall(f)[0]
        
    if re_email.match(f):
        email = re_email.findall(f)[0]
        email = email.lower()
        email_exist_flag = 1
    if len(f.split()) != 2:
        first_name = f
        last_name = ' '
    else:
        if re_name1.match(f):
            name = re_name1.findall(f)[0]
            first_name = name[0]
            last_name = name[1]
        elif re_name2.match(f): 
            name = re_name2.findall(f)[0]
            first_name = name[1]
            last_name = name[0]
        elif email_exist_flag == 1:
            pass
        else:
            print('error: cannot find name and email in f')
            print('f:', f)
            first_name = 'fake'
            last_name = f
            
    name = first_name + ' ' + last_name

    
    if email in named_email_list:
        idx = named_email_list.index(email)
        new_from_list.append(people_list[idx])
    elif name in name_list:
        idx = name_list.index(name)
        new_from_list.append(people_list[idx])
    elif email_exist_flag == 1:
        person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), email)
        new_from_list.append(person)
        unknown_idx = unknown_idx + 1
    else:
        person = (first_name, last_name, first_name + '_' + last_name)
        new_from_list.append(person)

error: cannot find name and email in f
f: 101 restaurant
error: cannot find name and email in f
f: cliﬀ thaell
error: cannot find name and email in f
f: 101 restaurant


In [90]:
len(new_from_list)

24837

In [91]:
len(from_list)

24837

In [92]:
len(new_from_list)

24837

In [93]:
new_to_lists = []

# replace the To section with unique people information

for ts in to_list: 
    ts = ts[2:-2].lower()
    ts = ts.split(';')
    new_to_list = []
    for t in ts:
        t = t.strip("'")
        email_exist_flag = 0
        email = ''
        first_name = ''
        last_name = ''
        if re_behalf.match(t):
            t = re_behalf.findall(t)[0]

        if re_email.match(t):
            email = re_email.findall(t)[0]
            email = email.lower()
            email_exist_flag = 1
        if len(t.split()) != 2:
            first_name = t
            last_name = ' '
        else:
            if re_name1.match(t):
                name = re_name1.findall(t)[0]
                first_name = name[0]
                last_name = name[1]
            elif re_name2.match(t): 
                name = re_name2.findall(t)[0]
                first_name = name[1]
                last_name = name[0]
            elif email_exist_flag == 1:
                pass
            else:
                first_name = 'fake'
                last_name = t
                print('error: cannot find name and email in t, make fake name')
                print('t:', t)
#                 break

        name = first_name + ' ' + last_name


        if email in named_email_list:
            idx = named_email_list.index(email)
            new_to_list.append(people_list[idx])
        elif name in name_list:
            idx = name_list.index(name)
            new_to_list.append(people_list[idx])
        elif email_exist_flag == 1:
            person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), email)
            new_to_list.append(person)
            unknown_idx = unknown_idx + 1
        else:
            person = (first_name, last_name, first_name + '_' + last_name)
            new_to_list.append(person)
    new_to_lists.append(new_to_list)

error: cannot find name and email in t, make fake name
t:  andersonjr., fred
error: cannot find name and email in t, make fake name
t:  impact100 grants
error: cannot find name and email in t, make fake name
t:  jeﬀ price
error: cannot find name and email in t, make fake name
t: * yourownutilities*
error: cannot find name and email in t, make fake name
t: sara 'saxner
error: cannot find name and email in t, make fake name
t:  101 managers
error: cannot find name and email in t, make fake name
t:  davidworrell (work)
error: cannot find name and email in t, make fake name
t:  ralph (businessfax)
error: cannot find name and email in t, make fake name
t:  fapsc1 .
error: cannot find name and email in t, make fake name
t:  tashdelmar .
error: cannot find name and email in t, make fake name
t:  rebeccagrimes (tal)
error: cannot find name and email in t, make fake name
t:  turbotrubo .
error: cannot find name and email in t, make fake name
t:  101 managers
error: cannot find name and email in

In [94]:
len(to_list)

24837

In [95]:
len(new_to_lists)

24837

In [96]:
new_to_lists[:10]

[[('brooks', 'hayes', 'brooks@culpeppercc.com')],
 [('kristen', 'coons', 'kristen.coons@talgov.com')],
 [('kim', 'rivers', 'kim@inkbridge.com')],
 [('alison', 'faris', 'alison.faris@talgov.com')],
 [('andrew', 'gillum', 'andrew.gillum@talgov.com')],
 [('allen', 'thompson', 'downtownmarket@earthlink.net')],
 [('michael', 'alfano', 'michael_alfano')],
 [('vince', 'long', 'longv@leoncounty')],
 [('chad', 'kittrell', 'chad@hunterandharp.com')],
 [('sue', 'dick', 'sdick@talchamber.com'),
  ('dana', 'noles', 'dnoles@talchamber.com'),
  ('whitney', 'weeks', 'whitney_weeks'),
  (' ed edward murray jr.', ' ', ' ed edward murray jr._ '),
  ('kristin', 'dozier', 'dozierk@leoncountyfl.gov'),
  ('cecilia', 'homison', 'chomison@firstcommercecu.org'),
  ('f2906', 'l2906', 'lonnie.ballard@talgov.com'),
  ('scott', 'balog', 'balogs@tcc.fl.edu'),
  ('kathy', 'bell', 'kgb@coloneybell.com'),
  ('reggie l. bouthillier', ' ', 'rbouthillier@stearnsweaver.com'),
  ('jt', 'burnette', 'jt@inkbridge.com'),
  ('w

In [97]:
new_cc_lists = []

# replace the cc section with unique people information

for ccs in cc_list: 
    ccs = ccs[2:-2].lower()
    ccs = ccs.split(';')
    new_cc_list = []
    if len(ccs) == 1 and ccs[0].strip() == '':
        new_cc_lists.append(new_cc_list)
        continue
    for cc in ccs:
        cc = cc.strip("'")
        email_exist_flag = 0
        email = ''
        first_name = ''
        last_name = ''
        if re_behalf.match(cc):
            cc = re_behalf.findall(cc)[0]

        if re_email.match(cc):
            email = re_email.findall(cc)[0]
            email = email.lower()
            email_exist_flag = 1
        if len(cc.split()) != 2:
            first_name = cc
            last_name = ' '
        else:
            if re_name1.match(cc):
                name = re_name1.findall(cc)[0]
                first_name = name[0]
                last_name = name[1]
            elif re_name2.match(cc): 
                name = re_name2.findall(cc)[0]
                first_name = name[1]
                last_name = name[0]
            elif email_exist_flag == 1:
                pass
            else:
                first_name = 'fake'
                last_name = cc
                print('error: cannot find name and email in cc, make fake name')
                print('cc:', cc)
#                 break

        name = first_name + ' ' + last_name


        if email in named_email_list:
            idx = named_email_list.index(email)
            new_cc_list.append(people_list[idx])
        elif name in name_list:
            idx = name_list.index(name)
            new_cc_list.append(people_list[idx])
        elif email_exist_flag == 1:
            person = ('f'+str(unknown_idx), 'l'+str(unknown_idx), email)
            new_cc_list.append(person)
            unknown_idx = unknown_idx + 1
        else:
            person = (first_name, last_name, first_name + '_' + last_name)
            new_cc_list.append(person)
    new_cc_lists.append(new_cc_list)

error: cannot find name and email in cc, make fake name
cc:  maisbel(mae) mendez
error: cannot find name and email in cc, make fake name
cc:  maisbel(mae) mendez
error: cannot find name and email in cc, make fake name
cc:  ramon.alexander@ myfloridahouse.gov
error: cannot find name and email in cc, make fake name
cc: adam' 'corey
error: cannot find name and email in cc, make fake name
cc:  'fastsigns 373
error: cannot find name and email in cc, make fake name
cc:  anneliese (or)
error: cannot find name and email in cc, make fake name
cc:  anneliese (or)
error: cannot find name and email in cc, make fake name
cc:  anneliese (or)
error: cannot find name and email in cc, make fake name
cc:  claudettecromartie (prsnl)
error: cannot find name and email in cc, make fake name
cc:  andersonjr., fred
error: cannot find name and email in cc, make fake name
cc:  briandavis (tpd)
error: cannot find name and email in cc, make fake name
cc: gary' 'yordon
error: cannot find name and email in cc, make

In [103]:
len(cc_list)

24837

In [104]:
len(new_cc_lists)

24837

In [105]:
new_cc_lists[:10]

[[('ryan', 'grindler', 'rgrindler@101tally.com'),
  ('david', 'rosenfeld', 'david.rosenfeld@talgov.com')],
 [('patrick', 'hurley', 'patrick_hurley')],
 [('jon', 'brown', 'brownjon@leoncountyfl.gov'),
  ('mathieu', 'cavell', 'cavellm@leoncountyfl.gov'),
  ('team', 'cmr', 'team_cmr'),
  ('victoria', 'connell', 'victoria_connell'),
  ('deborah', 'craig', 'deborah.craig@talgov.com'),
  ('stephanie', 'holloway', 'holloways@leoncountyfl.gov'),
  ('lindsay', 'jordan', 'lindsay_jordan'),
  ('angeline', 'taylor', 'angeline_taylor'),
  ('jay', 'townsend', 'jltownsend1986@gmail.com'),
  (' jessicamiller', ' ', ' jessicamiller_ ')],
 [('paige', 'carter', 'paige.tallahasseedowntown@gmail.com'),
  (' parade springtime tallahassee', ' ', ' parade springtime tallahassee_ '),
  ('edward', 'kring', 'edward.kring@talgov.com')],
 [('f21516', 'l21516', 'alan.williams@my')],
 [],
 [('dustin', 'daniels', 'r.daniels@gmail.com'),
  (' simpson, roxanne m', ' ', ' simpson, roxanne m_ ')],
 [('bryan', 'desloge', 

In [106]:
df_new = df.copy()
df_new['From'] = new_from_list
df_new['To'] = new_to_lists
df_new['CC'] = new_cc_lists
df_new


,From,Sent,To,CC,Subject,Attachments,Importance,Body,isDisplacement,isThread,isAutoMessage,hasAllCapLine,hasBadDate,filenm,mail_id
0,"(shane a. moniz, , smoniz@connandassociates.com)","['Friday, May 08, 2015 11:38 AM', 'March 2, 20...","[(brooks, hayes, brooks@culpeppercc.com)]","[(ryan, grindler, rgrindler@101tally.com), (da...","[""Edison RFI's""]",['RFI 008 Response.pdf; RFI 042 Response.pdf; ...,[],"['Response.pdf', 'Brooks, attached is our resp...",False,False,False,False,True,29142_fn_10-4-Cascade-2015-1-0_ln_42056.txt,29142
1,"(mark, beaudoin, mark.beaudoin@talgov.com)","['Tuesday, August 19, 2014 5:21 PM']","[(kristen, coons, kristen.coons@talgov.com)]","[(patrick, hurley, patrick_hurley)]",['RE: Revised lease'],[],[],['Good by me!'],False,False,False,False,False,41353_fn_10-3-Cascade-2014-2-0_ln_16811.txt,41353
2,"(ken, morris, morrisk@leoncountyfl.gov)","['Tuesday, May 28, 2013 1:22 PM']","[(kim, rivers, kim@inkbridge.com)]","[(jon, brown, brownjon@leoncountyfl.gov), (mat...",['Proposed Imagine Schedule'],[],[],"['Kim,', 'See comments in blue regarding the p...",False,False,False,False,False,01216_fn_17-2-IB2013-1-0_ln_44062.txt,1216
3,"(jennifer, naff, director@springtimetallahasse...","['Wednesday, March 15, 2017 5:32 PM']","[(alison, faris, alison.faris@talgov.com)]","[(paige, carter, paige.tallahasseedowntown@gma...",['RE: Springtime Parade'],[],[],"['Hi Alison,', 'Dont apologize, I know its a l...",False,False,False,True,False,32414_fn_32-1-PaigeCS-1-1_ln_18843.txt,32414
4,"(paul, lamaster, paullamaster250@comcast.net)","['Sunday, June 7, 2015 at 1:33:39 PM Eastern D...","[(andrew, gillum, andrew.gillum@talgov.com)]","[(f21516, l21516, alan.williams@my)]",['Tony Carvajal: Develop strategy to get ahead...,[],[],['Highly recommend you take a moment out of yo...,True,False,False,False,False,56710_fn_31-2-GaryYordon2-3_ln_41027.txt,56710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24832,"(gary, yordon, gary@zprgroup.com)","['Tuesday, September 29, 2015 at 10:53:38 PM E...","[(michelle, bono, michelle.bono@talgov.com)]",[],['Re: COCA Video'],[],[],['I understand. Traveling a bunch. Heading to ...,True,False,False,False,False,55898_fn_31-2-GaryYordon2-3_ln_9614.txt,55898
24833,"(paul, lamaster, paullamaster250@comcast.net)","['Wednesday, July 19, 2017 at 7:18:19 AM Easte...","[(william, haqield, whaqield@tallahassee.com)]",[],['Tallahassee Democrat E-Edi4on Ar4cle'],[],[],"['ron@sachsmedia.com, ray.bellamy@med.fsu.edu,...",True,False,False,False,False,42735_fn_31-2-GaryYordon2-1_ln_15355.txt,42735
24834,"(paul, lamaster, paullamaster250@comcast.net)","['Thursday, August 10, 2017 at 11:35:27 AM Eas...","[(andrew, gillum, andrew.gillum@talgov.com)]",[],['Tallahassee Democrat E-Edi4on Ar4cle'],[],[],"['nancy.miller@talgov.com, scoS.maddox@talgov....",True,False,False,False,False,42570_fn_31-2-GaryYordon2-1_ln_10355.txt,42570
24835,"(ryan, grindler, ryan@edisontally.com)","['Wednesday, February 22, 2017 at 6:27:34 PM E...","[(paige, carter, paige.tallahasseedowntown@gma...","[( steve adams, ashley edwards, allison scor, ...",['Re: Summer Movie Night Series Discussion Mee...,[],[],"['All,', 'We are s-ll working on some logis-ca...",True,False,False,False,False,36011_fn_32-1-PaigeCS-1-2_ln_5990.txt,36011


In [107]:
df

,From,Sent,To,CC,Subject,Attachments,Importance,Body,isDisplacement,isThread,isAutoMessage,hasAllCapLine,hasBadDate,filenm,mail_id
0,['Shane A. Moniz'],"['Friday, May 08, 2015 11:38 AM', 'March 2, 20...","['Brooks Hayes (brooks@culpeppercc.com)', 'Sha...",['Ryan Grindler (rgrindler@101tally.com); Rose...,"[""Edison RFI's""]",['RFI 008 Response.pdf; RFI 042 Response.pdf; ...,[],"['Response.pdf', 'Brooks, attached is our resp...",False,False,False,False,True,29142_fn_10-4-Cascade-2015-1-0_ln_42056.txt,29142
1,"['Beaudoin, Mark']","['Tuesday, August 19, 2014 5:21 PM']","['Coons, Kristen']","['Hurley, Patrick']",['RE: Revised lease'],[],[],['Good by me!'],False,False,False,False,False,41353_fn_10-3-Cascade-2014-2-0_ln_16811.txt,41353
2,['Ken Morris'],"['Tuesday, May 28, 2013 1:22 PM']",['kim@inkbridge.com'],['Jon Brown; Mathieu Cavell; CMR_Team CMR_Team...,['Proposed Imagine Schedule'],[],[],"['Kim,', 'See comments in blue regarding the p...",False,False,False,False,False,01216_fn_17-2-IB2013-1-0_ln_44062.txt,1216
3,['Jennifer Naff [mailto:director@springtimetal...,"['Wednesday, March 15, 2017 5:32 PM']","['Faris, Alison']",['Paige Carter-Smith; Parade Springtime Tallah...,['RE: Springtime Parade'],[],[],"['Hi Alison,', 'Dont apologize, I know its a l...",False,False,False,True,False,32414_fn_32-1-PaigeCS-1-1_ln_18843.txt,32414
4,['Paul LaMaster'],"['Sunday, June 7, 2015 at 1:33:39 PM Eastern D...","[' andrew.gillum@talgov.com, anita.ﬂowers@talg...","[' alan.williams@myﬂoridahouse.gov, gary@zprgr...",['Tony Carvajal: Develop strategy to get ahead...,[],[],['Highly recommend you take a moment out of yo...,True,False,False,False,False,56710_fn_31-2-GaryYordon2-3_ln_41027.txt,56710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24832,['Gary@zprgroup.com'],"['Tuesday, September 29, 2015 at 10:53:38 PM E...","[' Bono, Michelle']",[],['Re: COCA Video'],[],[],['I understand. Traveling a bunch. Heading to ...,True,False,False,False,False,55898_fn_31-2-GaryYordon2-3_ln_9614.txt,55898
24833,['Paul LaMaster'],"['Wednesday, July 19, 2017 at 7:18:19 AM Easte...","[' whaQield@tallahassee.com, bcoSerell@tallaha...",[],['Tallahassee Democrat E-Edi4on Ar4cle'],[],[],"['ron@sachsmedia.com, ray.bellamy@med.fsu.edu,...",True,False,False,False,False,42735_fn_31-2-GaryYordon2-1_ln_15355.txt,42735
24834,['Paul LaMaster'],"['Thursday, August 10, 2017 at 11:35:27 AM Eas...","[' andrew.gillum@talgov.com, cur4s.richardson@...",[],['Tallahassee Democrat E-Edi4on Ar4cle'],[],[],"['nancy.miller@talgov.com, scoS.maddox@talgov....",True,False,False,False,False,42570_fn_31-2-GaryYordon2-1_ln_10355.txt,42570
24835,['Ryan Grindler'],"['Wednesday, February 22, 2017 at 6:27:34 PM E...",[' Paige Carter-Smith'],"[' Steve Adams, Ashley Edwards, Allison ScoR, ...",['Re: Summer Movie Night Series Discussion Mee...,[],[],"['All,', 'We are s-ll working on some logis-ca...",True,False,False,False,False,36011_fn_32-1-PaigeCS-1-2_ln_5990.txt,36011


In [109]:
df_new.to_csv('newest_cleaned_output.csv', index=0)

In [121]:
df_new['filenm'].iloc[0]

'29142_fn_10-4-Cascade-2015-1-0_ln_42056.txt'

In [113]:
df_org=pd.read_excel('emails-combine 06162020.xlsx',sheet_name=0,usecols=[1,2,3,15,19,23])
gorvenment_people_list = []
for i in range(len(df_org)):
    row = df_org.iloc[i]
    if row['Organization1'] is not np.nan:
        if 'Tallahassee' in row['Organization1'] or 'Leon County' in row['Organization1']:
            gorvenment_people_list.append( (row['First'],row['Last'],row['Email']) )
            continue
    if row['Organization 2'] is not np.nan:
        if 'Tallahassee' in row['Organization 2'] or 'Leon County' in row['Organization 2']:
            gorvenment_people_list.append( (row['First'],row['Last'],row['Email']) )
            continue
    if row['Organization 3'] is not np.nan:
        if 'Tallahassee' in row['Organization 3'] or 'Leon County' in row['Organization 3']:
            gorvenment_people_list.append( (row['First'],row['Last'],row['Email']) )
            continue

In [114]:
df_cra=pd.read_excel('emails-combine 06162020.xlsx',sheet_name=1,usecols=[1,2])
df_cra = df_cra.iloc[2:16]
df_cra.columns = ['first','last']
df_cra = df_cra.applymap(str.lower)
df_cra = df_cra.reset_index(drop=True)


In [115]:
df_cra

,first,last
0,first,last
1,john,marks
2,john,dailey
3,george,desloge
4,jessica,miller
5,mark,mustian
6,phillip,sauls
7,gil,ziffer
8,andrew,gillum
9,nick,maddox


In [116]:
first = df_cra['first'].values.tolist()
last = df_cra['last'].values.tolist()

In [117]:
'carter' in first

False

In [310]:
df_new['Sent'] = datetime(df_new['Sent'].values.tolist())

# .dt.year

TypeError: an integer is required (got type list)